In [35]:
import pandas as pd
data = pd.read_csv("New_York_Times_labeled_dataframe_manual_cleaned.csv")
data = data[data["ranges"].isna() == False]

In [36]:
data

,Unnamed: 0,sentence,labels,ranges
0,0,"in 2008, hillary clinton was defeated in the d...","hillary clinton, barack obama, obama's, hillar...","[[9, 24], [68, 80], [224, 231], [320, 327], [4..."
1,1,"""the deeper reason putin seduces is that he be...",putin,"[[19, 24], [138, 143], [549, 554], [1033, 1038..."
4,4,i'll give ann coulter points for being forthri...,ann coulter,"[[10, 21]]"
7,7,you excuse trump’s tweets as less than artful ...,"trump’s, mr. comey","[[11, 18], [144, 151], [63, 72]]"
8,8,"quite frankly, i don't think it's too hard to ...",michael cohen,"[[55, 68]]"
...,...,...,...,...
491,491,"to answer the question, america will become ev...","trump, ginsburg, trump's","[[461, 466], [356, 364], [277, 284]]"
493,493,some of us don't pine for the dignity of a roy...,"obamas, mr. obama, george w. bush","[[149, 155], [322, 331], [379, 393]]"
495,495,"for some reasons, mr. trump has come to believ...","mr. trump, mcmaster,","[[18, 27], [511, 520], [591, 600], [940, 949],..."
496,496,to be an american citizen with the right to vo...,trump,"[[514, 519]]"


In [37]:
training_data = []

import json

training_test_text = []

training_test_entity = []

for index, row in data.iterrows():
    text = row["sentence"]
    range_word = json.loads(row["ranges"])
    temp_list = []
    
    if len(range_word) == 1:
        range_word = range_word[0]
        range_word.append("PER_CUSTOM")
        entity_dict ={"entities" : [tuple(range_word)]}
    else:
        for i, inner_range in enumerate(range_word):
            inner_range.append("PER_CUSTOM")
            temp_list.append(tuple(inner_range))
        range_word = temp_list
        entity_dict = {"entities" : tuple(range_word)}
    
    training_test_text.append(text)

    training_test_entity.append(entity_dict)
        
    
    

In [38]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_entity, test_entity = train_test_split(training_test_text, 
                                                                   training_test_entity, 
                                                                   test_size=0.2, shuffle=False)

In [39]:
train_entity[0]

{'entities': ((9, 24, 'PER_CUSTOM'),
  (68, 80, 'PER_CUSTOM'),
  (224, 231, 'PER_CUSTOM'),
  (320, 327, 'PER_CUSTOM'),
  (429, 436, 'PER_CUSTOM'),
  (515, 522, 'PER_CUSTOM'),
  (800, 807, 'PER_CUSTOM'),
  (447, 452, 'PER_CUSTOM'),
  (625, 630, 'PER_CUSTOM'),
  (764, 769, 'PER_CUSTOM'))}

In [40]:
search_for = "{'entities': ((16, 19, 'PER_CUSTOM'), (117, 120, 'PER_CUSTOM'))}"
for i, labels in enumerate(train_entity):
    if str(labels) == search_for:
        print("Found at", i)

In [41]:
train_text[68]

"i do believe it might. who would have thought that paula jones and then ms. lewinsky would be 'slick willy's' undoing. i believe the only reason clinton was not found guilty during his impeachment had to do with sexual indiscretions by other democrat senators, namely for starters ted kennedy and mr. lynda byrd johnson [chuck robb]."

In [42]:
import spacy
from spacy.training import Example

def create_pipeline():
    model = None
    if model is not None:
        nlp = spacy.load(model)
        print("Loads model '%s'" % model)
    else:
        nlp = spacy.blank("en")
        print("Created blank 'en' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        print("get pipeline")
        ner = nlp.get_pipe("ner")
    return ner, nlp

In [43]:
ner, nlp = create_pipeline()
n_iter = 6
loss_graph_list = []
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(other_pipes):
    optimizer = nlp.begin_training()
for itn in range(n_iter):
    losses = {}
    for text, annotations in zip(train_text, train_entity):
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses)
        
    print("Iteration", itn + 1, "Losses", losses)
    loss_graph_list.append(losses)

Created blank 'en' model


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "i do believe it might. who would have thought that..." with entities "((51, 62, 'PER_CUSTOM'), (72, 84, 'PER_CUSTOM'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "brooks is so incorrect in his analysis of trump an..." with entities "((0, 6, 'PER_CUSTOM'), (42, 47, 'PER_CUSTOM'), (96...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310

Iteration 1 Losses {'ner': 1305.7351313332958}
Iteration 2 Losses {'ner': 200.76379404405208}
Iteration 3 Losses {'ner': 81.84224185969514}
Iteration 4 Losses {'ner': 46.41342233154541}
Iteration 5 Losses {'ner': 42.99864474590217}
Iteration 6 Losses {'ner': 30.478588733891943}


In [80]:
#Uncomment to save comment
#import pickle
#with open("./models/custom_model_NYT.pickel", "wb") as f:
#    pickle.dump(nlp, f)

In [122]:
df_test = pd.DataFrame(columns=["sentence", "label"])

df_test["sentence"] = test_text
df_test["label"] = test_entity

df_test.to_csv("test_data_NYT.csv")

In [82]:
from spacy.scorer import Scorer

# Default scoring pipeline
scorer = Scorer()


In [1]:
class NERmodel:
    def __init__(self, data):
        self.data = data
        self.nlp = None
        
    def __create_pipeline():
        model = None
        if model is not None:
            nlp = spacy.load(model)
            print("Loads model '%s'" % model)
        else:
            nlp = spacy.blank("en")
            print("Created blank 'en' model")
        if 'ner' not in nlp.pipe_names:
            ner = nlp.add_pipe("ner")
        else:
            print("get pipeline")
            ner = nlp.get_pipe("ner")
        return ner, nlp
        
    def train(epochs):
        ner, nlp = __create_pipeline()
        n_iter = epochs
        other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
        with nlp.disable_pipes(other_pipes):
            optimizer = nlp.begin_training()
        for itn in range(n_iter):
            losses = {}
            for text, annotations in zip(train_text, train_entity):
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], losses=losses)

            print("Iteration", itn + 1, "Losses", losses)
            
    def save_model(modelpath):
        with open(modelpath, "wb") as f:
            pickle.dump(nlp, f)
    
    def load_model(modelpath):
        with open(modelpath, "rb") as f:
            nlp = pickle.load(f)